## Producer Script for Live-Data from API-Call

#### Imports

In [1]:
import json
import time
from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin import NewTopic
import requests
import datetime

In [2]:
admin = KafkaAdminClient(bootstrap_servers="localhost:9092")
producer = KafkaProducer(bootstrap_servers="localhost:9092")

### Try to create topics

In [3]:
if "preise" not in admin.list_topics() and "stationen" not in admin.list_topics():
    new_topics= list()
    new_topics.append(NewTopic(name="preise", num_partitions=1, replication_factor=1))
    new_topics.append(NewTopic(name="stationen", num_partitions=1, replication_factor=1))
    admin.create_topics(new_topics)

In [4]:
def add_price(date, uuid, diesel, e5,e10):
    j = json.dumps({"date": date, "station_uuid":uuid,"diesel":diesel,"e10":e10,"e5":e5})
    #print(j)
    producer.send("preise",str(j).encode())

def add_station( uuid, name, brand, street, house_number, post_code,city, lat, lng):
    j = json.dumps({"uuid":uuid,"name":name,"brand":brand,"street":street, "house_number":house_number,"post_code":post_code,"city":city,"latitude":lat,"longitude":lng,"first_active":"","openingtimes_json":None})
    #print(j)
    producer.send("stationen",str(j).encode())

#### API Call

In [5]:
API_KEY = "e82265a8-b634-ed81-69d2-82b1249c39ea"
BASE_URL = "https://creativecommons.tankerkoenig.de/json/list.php?"
LAT=49.495411
LNG=8.461562
RAD=10.0

def api_call():
    response = requests.get(BASE_URL+"lat="+str(LAT)+"&lng="+str(LNG)+"&rad="+str(RAD)+"&type=all&apikey="+API_KEY)
    if response.status_code==200 and response.json()["ok"] is True:
        response = response.json()
        results = response["stations"]
        print("Got "+str(len(results))+" results")
        for r in results:
            add_price(date=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S+01"), uuid=r["id"], diesel=r["diesel"], e5=r["e5"], e10=r["e10"])
            add_station(uuid=r["id"],name=r["name"],brand=r["brand"],street=r["street"], city=r["place"], house_number=r["houseNumber"], post_code=r["postCode"], lat=r["lat"],lng=r["lng"])

In [ ]:
try:
    while 1:
        api_call()
        time.sleep(60)
except KeyboardInterrupt:
    print("KeyBoardInterrupt")

Got 82 results
Got 82 results


Close connection to producer

In [ ]:
producer.close()